<a href="https://colab.research.google.com/github/KatherinneOlaya/Prompt-Engineering/blob/main/Day4_Agente_Kathe_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Documentación del Cuaderno de Colab

Este cuaderno documenta el proceso de configuración y ejecución de un agente que interactúa con la base de datos Supabase y la API de Groq para realizar un análisis de regresión lineal y generar una explicación con un modelo de lenguaje grande (LLM).

## Pasos Realizados:

1.  **Instalación de Librerías (`zJtHXx_5ESX5`)**:
    *   Se instalaron las librerías necesarias (`groq`, `supabase`, `python-dotenv`, `pandas`, `scikit-learn`) utilizando `pip`.
    *   Se añadió `%%capture` al inicio de la celda para ocultar la salida de la instalación.

2.  **Configuración de Variables de Entorno (`CLJxL95wGvJX`)**:
    *   Se importaron los módulos necesarios (`os`, `Groq`, `pandas`, `create_client`, `Client`).
    *   Se configuraron las variables de entorno `GROK_API`, `SUPABASE_URL` y `SUPABASE_KEY`.

3.  **Inicialización de Clientes (`C8v4ydlyPWqQ`)**:
    *   Se inicializaron los clientes de Groq (`client`) y Supabase (`supabase`) utilizando las claves y URLs configuradas en el paso anterior.
    *   Se corrigió un error inicial donde las variables de entorno no se estaban leyendo correctamente (se pasaban como cadenas literales en lugar de sus valores).

4.  **Definición de Funciones Utilitarias (`EIEPLTV3PawD`)**:
    *   `query_tool()`: Función para consultar datos de la tabla "productos" en Supabase.
    *   `clean_data(data)`: Función para limpiar los datos obtenidos, rellenando valores nulos con 0.
    *   `regression_lineal_train(df)`: Función que entrena un modelo de regresión lineal, prediciendo `precio_solo` a partir de `precio_mini`.
    *   `LLM(prompt)`: Función para interactuar con un modelo de lenguaje grande (LLM) de Groq.
        *   **Historial de Cambios y Errores del LLM**:
            *   Inicialmente, la función `LLM` fue definida incorrectamente dentro de `regression_lineal_train`, causando un `NameError`. Se corrigió moviéndola a la cabecera global.
            *   Debido a que varios modelos de Groq fueron reportados como 'decommissioned' (fuera de servicio) o 'model_not_found', se intentaron múltiples modelos:
                *   `mixtral-8x7b-32768` (inicial, luego se informó como fuera de servicio).
                *   `mixtral-8x7b-32k` (se informó como no encontrado/sin acceso).
                *   `llama3-8b-8192` (se informó como fuera de servicio).
                *   `llama3-70b-8192` (se informó como fuera de servicio).
                *   `mixtral-8x7b-32768` (se intentó de nuevo, se informó como fuera de servicio).
                *   `gemma-7b-it` (se informó como fuera de servicio).
                *   Finalmente, se configuró el modelo a `llama2-70b-4096`. (Basado en el último intento de modelo que se agregó a la celda).
            *   Se añadió un manejo de errores robusto a la función `LLM` para proporcionar mensajes más claros si un modelo no se encuentra, está fuera de servicio o hay problemas de acceso con la API key.

5.  **Implementación y Ejecución del Agente Principal (`PSii4mqtgWJ0`)**:
    *   Se definió la función `agente(prompt)` que orquesta los pasos:
        1.  Extrae datos (`query_tool`).
        2.  Limpia datos (`clean_data`).
        3.  Entrena el modelo de regresión (`regression_lineal_train`).
        4.  Genera una explicación de los resultados con el `LLM`.
    *   El agente fue ejecutado con el prompt "Analiza ventas y crea una regresión lineal", y se imprimieron los resultados.

In [1]:
%%capture
!pip install groq supabase python-dotenv pandas scikit-learn

In [6]:
import os
from groq import Groq
import pandas as pd
from supabase import create_client, Client

os.environ['GROK_API'] = ""
os.environ['SUPABASE_URL'] = "https://.supabase.co"
os.environ['SUPABASE_KEY'] = ""

In [7]:
client = Groq(api_key=os.environ.get('GROK_API'))
supabase = create_client(os.environ.get('SUPABASE_URL'), os.environ.get('SUPABASE_KEY'))

In [3]:
# ========================= Functions ===========================

def query_tool():
  data = supabase.table("productos").select("*").execute()
  return data.data

def clean_data(data):
  df = pd.DataFrame(data)
  df.fillna(0,inplace = True)
  return df

def regression_lineal_train(df):
  from sklearn.linear_model import LinearRegression

  df_clean = df.dropna(subset = ['precio_mini'])

  X = df_clean[['precio_mini']]
  y = df_clean['precio_solo']

  model = LinearRegression()
  model.fit(X, y)

  return {
        "coef": model.coef_.tolist(),
        "intercept": model.intercept_
    }

def LLM(prompt):
      response = client.chat.completions.create(
          model="llama-3.1-8b-instant",
          messages = [{"role": "user", "content": prompt}]
        )
      return response.choices[0].message.content


In [8]:
def agente(prompt):
    print("Thought: Necesito extraer datos…")
    print("#############################################################################################")

    print("\nAction: supabase_query")
    datos = query_tool()
    print("Observation:", datos)
    print("#############################################################################################")

    print("\nThought: Ahora limpio los datos…")
    df = clean_data(datos)
    print("Observation: Datos limpios")
    print("#############################################################################################")

    print("\nAction: train_regression")
    modelo = regression_lineal_train(df)
    print("Observation:", modelo)
    print("#############################################################################################")


    print("\nThought: generando explicación…")
    explicacion = LLM(f"""
Explica estos resultados de regresión lineal de acuerdo al negocio:
{modelo}
""")

    return explicacion

# ====== 5. Ejecutar agente ======
resultado = agente("Analiza ventas y crea una regresión lineal.")
print("\nFinal Answer:", resultado)

Thought: Necesito extraer datos…
#############################################################################################

Action: supabase_query
Observation: [{'id': 1, 'producto': 'WHOPPER', 'tipo': 'solo', 'precio_solo': 9400, 'precio_mini': 16100, 'precio_maxi': 18600, 'categoria': 'hamburguesa'}, {'id': 2, 'producto': 'WHOPPER ESPECIAL', 'tipo': 'solo', 'precio_solo': 14600, 'precio_mini': 21100, 'precio_maxi': 23600, 'categoria': 'hamburguesa'}, {'id': 3, 'producto': 'WHOPPER SUREÑA', 'tipo': 'solo', 'precio_solo': 12600, 'precio_mini': 19100, 'precio_maxi': 21000, 'categoria': 'hamburguesa'}, {'id': 4, 'producto': 'WHOPPER KING', 'tipo': 'solo', 'precio_solo': 20600, 'precio_mini': None, 'precio_maxi': None, 'categoria': 'hamburguesa'}, {'id': 5, 'producto': 'WHOPPER KING DE POLLO', 'tipo': 'solo', 'precio_solo': 21600, 'precio_mini': None, 'precio_maxi': None, 'categoria': 'hamburguesa'}, {'id': 6, 'producto': 'WHOPPER DE POLLO', 'tipo': 'solo', 'precio_solo': 14200, 'prec